In [1]:
import torch
import pandas as pd
import torch.nn as nn
from pandas import DataFrame


In [2]:
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv', index_col=0)

In [3]:
object_columns = train_df.dtypes[train_df.dtypes==object].index.to_list()
numeric_columns = train_df.dtypes[train_df.dtypes!=object].index.to_list()

In [4]:
train_df[numeric_columns] = train_df[numeric_columns].fillna(0)
train_df[object_columns] = train_df[object_columns].fillna('NA')

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in object_columns:
    le.fit(train_df[col])
    train_df[col] = le.transform(train_df[col])
    
train_df[object_columns].head(5)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,3,1,1,3,3,0,4,0,5,2,...,1,2,5,5,2,3,4,1,8,4
2,3,1,1,3,3,0,2,0,24,1,...,1,2,5,5,2,3,4,1,8,4
3,3,1,1,0,3,0,4,0,5,2,...,1,2,5,5,2,3,4,1,8,4
4,3,1,1,0,3,0,0,0,6,2,...,5,3,5,5,2,3,4,1,8,0
5,3,1,1,0,3,0,2,0,15,2,...,1,2,5,5,2,3,4,1,8,4


In [6]:
train_df = train_df.astype(float)

In [7]:
means, maxs, mins = dict(), dict(), dict()
for col in train_df:
    means[col] = train_df[col].mean()
    maxs[col] = train_df[col].max()
    mins[col] = train_df[col].min()
    
train_df = (train_df - train_df.mean()) / (train_df.max() - train_df.min())
train_df.describe()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1460.000000,1.460000e+03,...,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03
mean,-2.433366e-17,-4.197556e-17,1.155849e-17,-3.193792e-18,3.862968e-17,1.216683e-18,2.250863e-17,-3.528380e-17,0.000000,4.866731e-17,...,-9.125121e-19,3.361086e-17,2.007527e-17,-1.916275e-17,1.901067e-19,2.615868e-17,1.183589e-14,2.418157e-17,2.555034e-17,1.460019e-17
std,2.488269e-01,1.580044e-01,1.107486e-01,4.665342e-02,6.399614e-02,1.248334e-01,4.697187e-01,2.358886e-01,0.026171,4.056586e-01,...,5.444080e-02,4.690095e-02,2.707280e-01,9.302551e-02,3.200794e-02,2.457842e-01,3.320238e-01,1.940125e-01,2.201709e-01,1.103215e-01
min,-2.170427e-01,-7.571918e-01,-1.841000e-01,-4.308036e-02,-9.958904e-01,-4.969178e-01,-6.474886e-01,-9.257991e-01,-0.000685,-7.547945e-01,...,-3.738352e-03,-9.970320e-01,-8.761986e-01,-2.672945e-01,-2.805745e-03,-4.838107e-01,-4.539384e-01,-9.391267e-01,-7.541096e-01,-2.027791e-01
25%,-2.170427e-01,-7.191781e-03,-4.991466e-02,-1.385089e-02,4.109589e-03,3.082192e-03,-6.474886e-01,7.420091e-02,-0.000685,-2.547945e-01,...,-3.738352e-03,2.968037e-03,1.238014e-01,-1.729452e-02,-2.805745e-03,-1.201743e-01,-2.039384e-01,6.087329e-02,4.589041e-02,-7.074878e-02
50%,-4.057212e-02,-7.191781e-03,1.717799e-02,-4.853248e-03,4.109589e-03,3.082192e-03,3.525114e-01,7.420091e-02,-0.000685,2.452055e-01,...,-3.738352e-03,2.968037e-03,1.238014e-01,-1.729452e-02,-2.805745e-03,-2.926526e-02,4.606164e-02,6.087329e-02,4.589041e-02,-2.488709e-02
75%,7.707494e-02,-7.191781e-03,6.829621e-02,5.069863e-03,4.109589e-03,3.082192e-03,3.525114e-01,7.420091e-02,-0.000685,2.452055e-01,...,-3.738352e-03,2.968037e-03,1.238014e-01,-1.729452e-02,-2.805745e-03,1.525529e-01,2.960616e-01,6.087329e-02,4.589041e-02,4.593640e-02
max,7.829573e-01,2.428082e-01,8.159000e-01,9.569196e-01,4.109589e-03,5.030822e-01,3.525114e-01,7.420091e-02,0.999315,2.452055e-01,...,9.962616e-01,2.968037e-03,1.238014e-01,7.327055e-01,9.971943e-01,5.161893e-01,5.460616e-01,6.087329e-02,2.458904e-01,7.972209e-01


In [8]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df):
        self.data = df.values
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = torch.tensor(self.data[index, :-1])
        y = torch.tensor(self.data[index, -1:])
        return x, y

In [9]:
dataset = CustomDataset(train_df)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [10]:
class Net(nn.Module):
    def __init__(self, d_in, h1, h2, h3, d_out):
        super(Net, self).__init__()
        
        self.linear1 = nn.Linear(d_in, h1, dtype=float)
        self.linear2 = nn.Linear(h1, h2, dtype=float)
        self.linear3 = nn.Linear(h2, h3, dtype=float)
        self.linear4 = nn.Linear(h3, d_out, dtype=float)
        
    def forward(self, x):                
        y_pred = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(y_pred).clamp(min=0)
        y_pred = self.linear3(y_pred).clamp(min=0)
        y_pred = self.linear4(y_pred)
        return y_pred

In [11]:
h1, h2, h3 = 500, 1000, 200
d_in, d_out = train_df.shape[1] - 1, 1

In [12]:
model1 = Net(d_in, h1, h2, h3, d_out)
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model1.parameters(), lr=1e-2)

In [13]:
epoch_count = 500
for epoch in range(epoch_count):
    loss_sum = 0.0
    for batch in dataloader:
        x_tensor, y_tensor = batch
        y_pred = model1(x_tensor)        
        loss = criterion(y_pred, y_tensor)            
        loss_sum += loss.item()

        if torch.isnan(loss):
            break

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(epoch, loss_sum)        

0 11.661913941662656
1 5.42398138913041
2 4.269584870185571
3 3.726175782894273
4 3.4127132417561126
5 3.3273389662333908
6 3.117674197836294
7 2.953513915985354
8 2.8225934472539818
9 2.771141131785816
10 2.7179603165561144
11 2.4171881199545306
12 2.412710976118788
13 2.3928545836611637
14 2.3679537200284377
15 2.321600720794496
16 2.269712862694277
17 2.268649850635066
18 2.0890395399903787
19 2.0485338177081776
20 2.0757648530277093
21 1.858652650353776
22 1.9140237971993077
23 1.9280151388548223
24 1.8794646955861414
25 1.895599290738658
26 1.618768764690384
27 1.62526008526995
28 1.724285860955128
29 1.6250640910756107
30 1.5946027427827885
31 1.529736906222407
32 1.4725707250019673
33 1.4184773763284149
34 1.4453199314947553
35 1.373764662151323
36 1.6550207728819413
37 1.3037461595016893
38 1.4380325426840501
39 1.4980254385896683
40 1.3292145197482164
41 1.30008103918951
42 1.042563771994487
43 1.1914218320840184
44 1.04318719752188
45 0.9992405379405914
46 1.0161823918699795


In [14]:
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv', index_col=0)

In [15]:
numeric_columns.remove('SalePrice')
test_df[numeric_columns] = test_df[numeric_columns].fillna(0)
test_df[object_columns] = test_df[object_columns].fillna('NA')

In [16]:
for col in object_columns:
    le.fit(test_df[col])
    test_df[col] = le.transform(test_df[col])
    
test_df[object_columns].head(5)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,3,1,1,3,3,0,4,0,12,1,...,1,3,4,5,2,2,2,1,9,4
1462,4,1,1,0,3,0,0,0,12,2,...,1,3,4,5,2,2,4,0,9,4
1463,4,1,1,0,3,0,4,0,8,2,...,1,0,4,5,2,2,2,1,9,4
1464,4,1,1,0,3,0,4,0,8,2,...,1,0,4,5,2,2,4,1,9,4
1465,4,1,1,0,1,0,4,0,22,2,...,1,2,4,5,2,2,4,1,9,4


In [17]:
train_df = train_df.astype(float)

In [18]:
x_tensor = torch.tensor(test_df.iloc[:, :].values, dtype=float)
x_tensor.shape

torch.Size([1459, 79])

In [19]:
y_tensor = model1(x_tensor)
y_tensor.shape

torch.Size([1459, 1])

In [20]:
with torch.no_grad():
    outputs = model1(x_tensor)    
    result = pd.DataFrame({'Id': test_df.index, 'SalePrice': outputs.reshape(outputs.shape[0])})
    result['SalePrice'] = result['SalePrice'] * (maxs['SalePrice'] - mins['SalePrice']) + means['SalePrice']    
    result.to_csv('sub.csv', index=False)
    